In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
def play_rps(p1Choice, p2Choice):
    
    result = 0
    
    if p1Choice != p2Choice:
        if p1Choice == 'r':
            if p2Choice == 'p':
                result = 2
            elif p2Choice == 's':
                result = 1
        elif p1Choice == 'p':
            if p2Choice == 'r':
                result = 1
            elif p2Choice == 's':
                result = 2
        elif p1Choice == 's':
            if p2Choice == 'r':
                result = 2
            elif p2Choice == 'p':
                result = 1
    
    else:
        result = 0
        
    return result

In [3]:
choices = ['p', 'r', 's']

p1Choices = np.random.choice(choices, 10000).tolist()

p2Choices = np.random.choice(choices, 10000).tolist()

p1Scores = [play_rps(p1Choices[x], p2Choices[x]) for x in range(len(p1Choices))]

rps_games = {
    'p1Choices': p1Choices,
    'p2Choices': p2Choices,
    'game_results': p1Scores
}

rps_games = pd.DataFrame(data=rps_games)

rps_games.head()

#train_data = np.append(p1Choices, p2Choices, axis=1)

,p1Choices,p2Choices,game_results
0,s,r,2
1,p,r,1
2,r,r,0
3,r,p,2
4,s,p,1


In [4]:
labelEnc = LabelEncoder()

train_data = pd.DataFrame()

for c in rps_games.columns:
    train_data[c]=labelEnc.fit_transform(rps_games[c])
    
train_data.head()

,p1Choices,p2Choices,game_results
0,2,1,2
1,0,1,1
2,1,1,0
3,1,0,2
4,2,0,1


In [7]:
#X_train, X_test, y_train, y_test = train_test_split(train_data, p1Scores, test_size=0.33, random_state=42)

train = train_data.sample(frac=0.8, random_state=0)
test = train_data.drop(train.index)

# train_labels = train.pop('game_results')
# test_labels = test.pop('game_results')

train = train.to_numpy()
test = test.to_numpy()

# train_labels = train_labels.to_numpy()
# test_labels = test_labels.to_numpy()

In [8]:
print(train)

[[1 1 0]
 [1 1 0]
 [0 2 2]
 ...
 [0 2 2]
 [1 0 2]
 [1 0 2]]


# Model Training

## Artificial Neural Network

In [6]:
snn = tf.keras.models.Sequential([
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

snn.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [8]:
snn.fit(train, train_labels, epochs=10)

Train on 8000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 149us/sample - loss: 1.1146 - accuracy: 0.2076
Epoch 2/10
8000/8000 [==============================] - 0s 38us/sample - loss: 0.9997 - accuracy: 0.4234
Epoch 3/10
8000/8000 [==============================] - 0s 44us/sample - loss: 0.8843 - accuracy: 0.4970
Epoch 4/10
8000/8000 [==============================] - 0s 32us/sample - loss: 0.7767 - accuracy: 0.6755
Epoch 5/10
8000/8000 [==============================] - 0s 33us/sample - loss: 0.6708 - accuracy: 0.8239
Epoch 6/10
8000/8000 [==============================] - 0s 32us/sample - loss: 0.5685 - accuracy: 0.9275
Epoch 7/10
8000/8000 [==============================] - 0s 34us/sample - loss: 0.4754 - accuracy: 0.9865
Epoch 8/10
8000/8000 [==============================] - 0s 33us/sample - loss: 0.3874 - accuracy: 1.0000
Epoch 9/10
8000/8000 [==============================] - 0s 36us/sample - loss: 0.3089 - accuracy: 1.0000
Epoch 10/10
8000/8000 [=========

In [9]:
# print(test_dataset[0:5])

# model.predict(test_dataset).flatten()

## Simple RNN

RNN model required three-dimentional input data so we must transform the current data to reflect three dimensions

In [10]:
print(train)

num_epochs = 100


step = 4

#Add step elements into train and test

rnn_train = np.append(train, np.repeat(train[-1], step))
rnn_test = np.append(test, np.repeat(test[-1], step))

print(rnn_train)

[[2 2]
 [2 0]
 [0 0]
 ...
 [1 0]
 [0 0]
 [1 2]]
[2 2 2 ... 2 2 2]


In [11]:
def convertToMatrix(data, step):
    X, Y = [], []
    for i in range(len(data)-step):
        d = i + step
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

trainX, trainY = convertToMatrix(train, step)
testX, testY = convertToMatrix(test, step)

In [12]:
#print(trainX)
#print(train[-1],trainX)

rnn_trainX = np.reshape(trainX, (trainX.shape[0], 2, trainX.shape[1]))
rnn_testX = np.reshape(testX, (testX.shape[0], 2, testX.shape[1]))

print(rnn_trainX.shape)

(7996, 2, 4)


In [16]:
rnn = tf.keras.Sequential()
rnn.add(keras.layers.SimpleRNN(units=32, input_shape=(2, step), activation="relu"))
rnn.add(keras.layers.Dense(8, activation='relu'))
rnn.add(keras.layers.Dense(3, activation='softmax'))

rnn.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

rnn.fit(rnn_trainX, trainY[:, 0], epochs=50)

Train on 7996 samples
Epoch 1/50
7996/7996 [==============================] - 2s 280us/sample - loss: 1.1020 - accuracy: 0.3198
Epoch 2/50
7996/7996 [==============================] - 1s 78us/sample - loss: 1.0986 - accuracy: 0.3354
Epoch 3/50
7996/7996 [==============================] - 1s 76us/sample - loss: 1.0981 - accuracy: 0.3348
Epoch 4/50
7996/7996 [==============================] - 1s 66us/sample - loss: 1.0977 - accuracy: 0.3405
Epoch 5/50
7996/7996 [==============================] - 1s 73us/sample - loss: 1.0974 - accuracy: 0.3444
Epoch 6/50
7996/7996 [==============================] - 1s 66us/sample - loss: 1.0966 - accuracy: 0.3460
Epoch 7/50
7996/7996 [==============================] - 1s 70us/sample - loss: 1.0961 - accuracy: 0.3512
Epoch 8/50
7996/7996 [==============================] - 1s 66us/sample - loss: 1.0955 - accuracy: 0.3519
Epoch 9/50
7996/7996 [==============================] - 1s 66us/sample - loss: 1.0950 - accuracy: 0.3558
Epoch 10/50
7996/7996 [=========

In [17]:
test = np.array([[[0, 1, 1, 1], [1, 2, 2, 2]]])
test = test.astype(float)
print(test.shape)
print(rnn.predict(test))

(1, 2, 4)
[[0.35419765 0.35171244 0.29408988]]
